In [1]:
# Used to track the request processing time
import time

from textwrap import dedent
from langchain_community.llms import Ollama
from crewai import Crew, Agent, Task

In [2]:
# Initialize Llama3 LLM for agents.
# This will access local models in the standard installation directory

llm = Ollama(model="NEW_MODEL")

In [3]:
# Uncomment the below line to test if the loaded model
# works and is indeed a custom model.

# llm.invoke("Tell me a joke")

# Create Agents

In [4]:
class FunctionTestAgents:

    def variable_name_agent(self):
        return Agent(
            role='Variable Name Agent',
            goal='Ensure that variable names are descriptive, concise, and accurate.',
            backstory=dedent(
                '''
                You're an expect agent that helps developers write clean, efficient code.
                Your job is to ensure that variable names are descriptive, concise, and accurate.
                Do not add comments to the code.
                '''),
            allow_delegation=False,
            verbose=True,
            llm=llm
    )

    def code_comment_agent(self):
        return Agent(
            role='Code Comment Agent',
            goal='Add comments to code to explain its purpose and functionality. Only add comments that are necessary for a user to understand the code.',
            backstory=dedent(
                '''
                You're an expect code commentor. Your job is to add comments to the code to explain its purpose and functionality. Only add comments that
                are necessary for a user to understand the code. You're aware that code should be self-explanatory whenever possible, and that comments
                should only be used to explain complex or non-obvious parts of the code.
                '''),
            allow_delegation=False,
            verbose=True,
            llm=llm
    )

    def documentation_agent(self):
        return Agent(
            role='Documentation Agent',
            goal='Document code in a way that is easy to understand and follow.',
            backstory=dedent(
                '''
                You're an expert in writing docstrings. Your documentation is clear, concise, accurate, and easy to understand.
                '''),
            allow_delegation=False,
            verbose=True,
            llm=llm
    )

# Create Tasks

In [5]:
class FunctionTestTasks:

    def improve_variable_names(self, agent, request):
        return Task(
            description=dedent(
                f'''
                Receive Python code. Ensure that all variable names are descriptive, concise, and accurate.
                Do not add comments to the code.

                Code
                ----
                {request}
                '''),
            expected_output=dedent(
                '''
                Python code with improved variable names
                '''),
                agent=agent
        )

    def add_code_comments(self, agent, request):
        return Task(
            description=dedent(
                f'''
                Receive Python code. Add comments to the code to explain its purpose and functionality.
                Only add comments that are necessary for a user to understand the code. If a line of
                code is self-explanatory, do not add a comment.

                Code
                ----
                {request}
                '''),
            expected_output=dedent(
                '''
                Python code with added comments
                '''),
                agent=agent
        )

    def review_code_comments(self, agent, request):
        return Task(
            description=dedent(
                f'''
                Receive Python code with comments. Review the code and ensure that all comments are
                accurate and helpful. If a comment is not necessary, remove it.

                Code
                ----
                {request}
                '''),
            expected_output=dedent(
                '''
                Python code with reviewed comments
                '''),
                agent=agent
        )

    def add_documentation(self, agent, request):
        return Task(
            description=dedent(
                f'''
                Add docstrings to the code to explain its purpose and functionality.
                Ensure the documentation is clear, concise, accurate, and easy to understand.

                Code
                ----
                {request}
                '''),
            expected_output=dedent(
                '''
                Python code with added documentation
                '''),
            output_file='updated_functions.py',
            agent=agent
        )

# Processing Functions
Start reading a file of functions, each preceeded by a comment explaining what the function is; extract the function and feed it through the LLM; then write the result to a new file.

In [6]:
# Define relative file path
test_suite_file_path = 'test_suite/test_functions_raw_practice.py'

# Save all captured functions to a list
functions = []

with open(test_suite_file_path, 'r', encoding='utf-8') as file:

    function_code      = ''
    capturing_function = False
    
    for line in file:
        
        # Check if the line is blank
        if line.strip() == '':
            
            # If the line is blank and we're capturing
            # a function, we've reached the end of the
            # current function; reset
            if capturing_function:
                functions.append(function_code.strip())
                function_code = ''
                capturing_function = False
        
        # If line isn't blank, start capturing function
        else:
            function_code += line
            capturing_function = True
    
    # If the file doesn't end in a blank line,
    # ensure the last function is saved.
    if capturing_function:
        functions.append(function_code.strip())

# Main

In [7]:
# Initialize Agents, Tasks
agents = FunctionTestAgents()
tasks = FunctionTestTasks()


# CREATE AGENTS
variable_name_agent = agents.variable_name_agent()
# code_comment_agent = agents.code_comment_agent()
documentation_agent = agents.documentation_agent()

print("##################################")
print("## Welcome to the Function Crew ##")
print("##################################")

# CREATE TASKS

updated_functions = []

for func in functions:

    improve_variable_names_task = tasks.improve_variable_names(variable_name_agent, func)
    # add_code_comments_task = tasks.add_code_comments(variable_name_agent, improve_variable_names_task)
    add_documentation_task = tasks.add_documentation(variable_name_agent, improve_variable_names_task)


    # CREATE CREW & ASSIGN TASKS
    crew = Crew(
        agents = [
            variable_name_agent,
            documentation_agent
        ],
        tasks = [
            improve_variable_names_task,
            add_documentation_task
        ],
        verbose=True
    )

    # Start code-improvement process
    start_time = time.perf_counter()
    output = crew.kickoff()
    end_time = time.perf_counter()
    
    updated_functions.append(output.strip())

    print("################################")
    print("## Refactored Function Output ##")
    print("################################")
    print(f">>> Time taken: {end_time - start_time:.2f} seconds <<<\n")
    
    
# WRITE FUNCTIONS TO FILE FOR REVIEW

# Use to write a file with the current datetime
import datetime
import os
    
# Check if directory exists; if not, create it
# This directory is relative to the location of this Jupyter Notebook
save_location_file_path = 'test_outputs'
if not os.path.exists(save_location_file_path):
    os.makedirs(save_location_file_path)

# Create file to write output

# Create custom datetime output, used for filename
current_datetime = datetime.datetime.now().strftime('%y-%m-%d_%H:%M:%S') + '.txt' 
output_filename  = 'function_output_' + current_datetime
full_file_path   = os.path.join(save_location_file_path, output_filename)

with open(full_file_path, 'w', encoding='utf-8') as f:
    for func in updated_functions:
        
        # Remove backticks and any remaining whitespaces from function
        cleaned_func = func.strip('`').strip()
        f.write(cleaned_func + "\n\n")
            
print(f'Functions successfully written to file "{full_file_path}".')

##################################
## Welcome to the Function Crew ##
##################################
 [DEBUG]: == Working Agent: Variable Name Agent
 [INFO]: == Starting Task: 
            Receive Python code. Ensure that all variable names are descriptive, concise, and accurate.
            Do not add comments to the code.

            Code
            ----
            def f(n):
a, b = 0, 1
for _ in range(n):
    a, b = b, a + b
return a



> Entering new CrewAgentExecutor chain...
Thought: I now can give a great answer

Final Answer:
```
def fibonacci(n):
    current_value, next_value = 0, 1
    for _ in range(n):
        current_value, next_value = next_value, current_value + next_value
    return current_value
```

> Finished chain.
 [DEBUG]: == [Variable Name Agent] Task output: ```
def fibonacci(n):
    current_value, next_value = 0, 1
    for _ in range(n):
        current_value, next_value = next_value, current_value + next_value
    return current_value
```


 [DEBUG]: == 

2024-05-16 12:26:16,478 - 8450341568 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


I now can give a great answer!

Final Answer:
```
def fibonacci(n):
    """
    Calculate the nth Fibonacci number.

    Args:
        n (int): The position of the Fibonacci number to calculate.

    Returns:
        int: The nth Fibonacci number.
    """
    current_value, next_value = 0, 1
    for _ in range(n):
        current_value, next_value = next_value, current_value + next_value
    return current_value
```

> Finished chain.
 [DEBUG]: == [Variable Name Agent] Task output: ```
def fibonacci(n):
    """
    Calculate the nth Fibonacci number.

    Args:
        n (int): The position of the Fibonacci number to calculate.

    Returns:
        int: The nth Fibonacci number.
    """
    current_value, next_value = 0, 1
    for _ in range(n):
        current_value, next_value = next_value, current_value + next_value
    return current_value
```


################################
## Refactored Function Output ##
################################
>>> Time taken: 28.81 seconds <<<

 [D